## 0. Libarary 불러오기 및 경로설정

In [1]:
import os
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import wandb
from torchvision import models
from torchvision import transforms
from PIL import Image 
from torchvision import models
from resnest.torch import resnest50
import timm

os.chdir('/opt/ml')
from utils.Dataset import Class18Dataset, cfg, get_img_paths
from utils.utils import calculate_norm, linear_param_initialize, train_mean_std
import torch.utils.data as data
from tqdm.notebook import tqdm
from train import train
from baseline.loss import FocalLoss, LabelSmoothingLoss, F1Loss
import warnings
from model.model import ThreeOneNet, Resnest50
from utils.utils import eval_mean_std
from baseline.dataset_mike import MaskBaseDataset, MyCrop
from model.Ensemble_model.baty import baty_model
from model.Ensemble_model.mike import mike_model
from model.Ensemble_model.peachy import peachy_model
from baseline.dataset_v2 import MaskBaseDataset, MyCrop, MyCropMid, MyCropDown, Ensemble
warnings.filterwarnings('ignore')

In [2]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 1. Model 정의

In [3]:
mike_model1 = mike_model.get_model(2)
mike_model2 = mike_model.get_model(2)
mike_model3 = mike_model.get_model(2)


mike_model1.load_state_dict(torch.load('/opt/ml/model/Ensemble_model/mike/2.pt'))
mike_model2.load_state_dict(torch.load('/opt/ml/model/Ensemble_model/mike/1.pt'))
mike_model3.load_state_dict(torch.load('/opt/ml/model/Ensemble_model/mike/0.pt'))


baty_model4 = baty_model.Resnext50()
baty_model5 = baty_model.Resnext50()

                                             
baty_model4.load_state_dict(torch.load('/opt/ml/model/Ensemble_model/baty/best3.pth'))
baty_model5.load_state_dict(torch.load('/opt/ml/model/Ensemble_model/baty/last.pth'))

<All keys matched successfully>

## 2. Test Dataset 정의

In [4]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform=None):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [5]:
batch_size = 100
device = 'cuda'

In [7]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]


transform = transforms.Compose([
                    MyCrop(),
                    transforms.Resize((370,324),Image.BILINEAR),
                    transforms.ToTensor(),
                    transforms.Grayscale(3)
                    ])

dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model1 = mike_model1.cuda()
model2 = mike_model2.cuda()
model3 = mike_model3.cuda()

model4 = baty_model4.cuda()
model5 = baty_model5.cuda()



model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions1 = []
for images in tqdm(loader, leave=False):
    with torch.no_grad():
        images = images.to(device)
        pred = model1(images) + model2(images) + model3(images) + model4(images) + model5(images)
        pred =pred.argmax(dim=-1)
    all_predictions1.extend(pred.cpu().numpy())

In [8]:
submission['ans'] = all_predictions1
submission.to_csv(os.path.join('/opt/ml/submission', 'submission-(baty3,last + mike0,1,2.csv)-nosoftmax-최종.csv'), index=False)
print('test inference is done!')

test inference is done!


In [15]:
all_predictions1[6473]

2

In [ ]:
1,3 제외